 # Récupération Donnée API Github

Chargement de la clé github 

In [1]:
import os
from dotenv import load_dotenv

load_dotenv("../../.env")

CLE_API_GITHUB = os.getenv('CLE_API_GITHUB')

In [2]:
import requests
import random
import base64
import time

HEADERS = {"Authorization": f"token {CLE_API_GITHUB}"}

def get_repos():
    # Recherche générale avec le mot "a" (ou autre lettre)
    url = "https://api.github.com/search/repositories"
    params = {
        "q": "a", # recherche avec la lettre "a"
        "order": "desc",
        "per_page": 5, 
        "page": random.randint(1, 100), # page random entre 1 et 100
        }  
    response = requests.get(url, headers=HEADERS, params=params)
    data_repos = []
    
    if response.status_code == 200:
        data = response.json()
        repos = data.get("items", [])
        if repos:
            for repo in repos:
                if(fetch_and_check_readme(repo)):
                    data_repos.append({
                        "name": repo["name"],
                        "html_url": repo["html_url"],
                        "description": repo["description"],
                        "readme" : repo["readme_content"]
                    })
            return data_repos
        else:
            return "Aucun dépôt trouvé."
    else:
        if response.status_code == 403:
            time.sleep(10)
            return get_repos()
        else :
            return f"Erreur: {response.status_code}, {response.text}"


def fetch_and_check_readme(repo):
    readme_url = repo["url"] + "/readme"
    response = requests.get(readme_url, headers=HEADERS)
    readme_is_present = False
    if response.status_code == 200:
        data = response.json()
        if "content" in data:
            # Décoder le contenu encodé en base64
            content = base64.b64decode(data["content"]).decode("utf-8").strip()
            if content:  # Vérifier que le README n'est pas vide
                repo["readme_content"] = content
                readme_is_present = True
    return readme_is_present


In [3]:
from kafka import KafkaProducer
import json
import random
from typing import Dict, Any

add_to_es_topic = "add_to_es"

"""Create a Kafka producer with default settings."""
def create_producer() -> KafkaProducer:
    return KafkaProducer(
        bootstrap_servers='localhost:29092',
        value_serializer=lambda x: json.dumps(x).encode('utf-8'),
        key_serializer=lambda k: k.encode('utf-8')
    )

def send_message(producer: KafkaProducer, topic: str, key: str, message: Dict[str, Any]) -> None:
    """Send a message to a Kafka topic."""
    try:
        future = producer.send(topic, key=key, value=message)
        metadata = future.get(timeout=5)
        print(f"Message sent - Key: {key}, Partition: {metadata.partition}, Offset: {metadata.offset}")
    except Exception as e:
        print(f"Error sending message: {repr(e)}")


def produce_messages(message, topic):
    producer = create_producer()
    
    unique_ids = ['id1', 'id2', 'id3', 'id4']    
    try:
        key = random.choice(unique_ids)
        send_message(producer, topic, key, message)
    except Exception as e:
        print(f"Failed to produce message: {str(e)}")
    finally:
        producer.close()

In [ ]:
from kafka import KafkaProducer
import json

def find_and_send_repos():
    repo = get_repos()
    for r in repo:
        print(f"Envoi du repo {r['name']} à Kafka")
        produce_messages(json.dumps(repo, indent=4), "add_to_es")

find_and_send_repos()

Envoi du repo a-react-app-example à Kafka
Error sending message: KafkaTimeoutError('Timeout after waiting for 5 secs.')
